Comrades, you knew it was coming. No more tricks, filters and hand-tuning. We feed our data into the deep learning sausage factory.

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.optimizers import SGD

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
import sys
import glob
import PIL.Image as Image
from os.path import basename
sys.path.append("../")
from putzlib import pieces,piecenames,piecenamesrev

## The training Set

50 chessboard images taken from scanned books, including the era prior to  computer typesetting. The file `training.py` contains code to separate images into directories containing light and dark squares, with file names labelling the piece. 

For machine learning, it makes sense to have the proportions of the categories in the training set, as close to the real world as we can. However this is might lead to certain categories being very underrepresented. **Example**: the white king is almost always at **e1** or **g1** which are both dark squares, and a similar situation holds for the black king. 

To alleviate this problem and increase robustness, I augmented the training set by about 8x by applying random translations. 

In [ ]:
def randomtrans(ima,maxoff=5):
    '''Randomly translate an array by (x,y) where x and y lie in (-maxoff,maxoff)'''
    l,w = ima.shape
    imc = np.zeros((l+2*maxoff,w+2*maxoff),dtype=ima.dtype)
    tx,ty = random.randint(0,2*maxoff),random.randint(0,2*maxoff)
    imc[tx:tx+l,ty:ty+w] = ima
    return imc[maxoff:maxoff+l,maxoff:maxoff+w]

## The network

The architecture is the now standard alternation of convolution-pooling-dropout layers. The notable feature is the omission of a final dense layer, in favour of global pooling with the appropriate number of channels (13 for each of the pieces and a blank square). 

Let us thank the [alchemists](https://www.youtube.com/watch?v=Qi1Yry33TQE) who discovered that this graph works just as well, with fewer parameters.

In [2]:
modela = Sequential()

modela.add(Dropout(0.2,input_shape=(32, 32,1)))
modela.add(Conv2D(32, (5, 5), activation='relu', padding="same"))
modela.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2)))
modela.add(Dropout(0.5))

modela.add(Conv2D(64, (5, 5), activation='relu',padding="same"))
modela.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2)))
modela.add(Dropout(0.5))

modela.add(Conv2D(64, (3, 3), activation='relu',padding="same"))
modela.add(Conv2D(64, (1, 1), activation='relu',padding="same"))
modela.add(Conv2D(13, (1, 1), activation='relu',padding="same"))

modela.add(GlobalAveragePooling2D())
modela.add(Activation('softmax'))

In [3]:
modela.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 7, 7, 64)          0         
__________

In [9]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

from keras.utils import plot_model
plot_model(modela, to_file='modela.png')

In [8]:
def makedataset(path,piecenums=range(13)):
    """Load squares and read labels from the filename
      return both images and labels"""
    # Each tile is a 32x32 grayscale image, add extra axis for numchannels=1
    imfiles=[]
    for p in piecenums:
        imfiles += glob.glob(path+piecenames[pieces[p]]+'*png')
    ims =  np.zeros([len(imfiles), 32, 32, 1], dtype=np.uint8)
    labels = np.zeros([len(imfiles), 13], dtype=np.float32)
    for i, impath in enumerate(imfiles):
        if i % 100 == 0:
            print(".",end=' ')   
    # Image
        ims[i,:,:,0] = np.asarray(Image.open(impath), dtype=np.uint8)
    # Label
        _lab = np.zeros(13, dtype=np.uint8)
        _ptype = basename(impath)[:2]
        _lab[pieces.index(piecenamesrev[_ptype])] = 1
        labels[i,:] = _lab
    print("Done")
    return ims,labels

In [14]:
ims,labels=makedataset('C:/Users/klein/Documents/putzdata/boardscomb3/darksquares/',range(13))
#shuffle order
s=np.arange(ims.shape[0])
np.random.shuffle(s)
ims,labels = ims[s],labels[s]

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Done


In [ ]:
modela.fit(train_images,train_labels, batch_size=100, epochs=15)

In [ ]:
modela.evaluate(test_images, test_labels, batch_size=100)

In [ ]:
from keras.models import load_model

modlight = load_model('../saved_models/lightA.h5')
moddark  = load_model('../saved_models/darkA.h5')

In [ ]:
def boardprednn2(squares,modlight,moddark):
    if len(squares) != 64:
        return ""
    sqresized = [cv2.resize(im,(32,32),interpolation = cv2.INTER_CUBIC) for im in squares]
    sqdark = np.zeros((32,32,32,1),np.float32)
    sqlight = np.zeros((32,32,32,1),np.float32)
    for i,s in enumerate(sqresized):
        ld = (i + i//8)%2
        if ld == 0:
            sqlight[i//2,:,:,0] = (1/255.0)*np.float32(s)
        else:
            sqdark[i//2,:,:,0] = (1/255.0)*np.float32(s)
    predslight = np.argmax(modlight.predict(sqlight),axis=1)
    predsdark = np.argmax(moddark.predict(sqdark),axis=1)
    bb = ""
    for i in range(64):
        ld = (i + i//8)%2
        pred = (predslight,predsdark)[ld][i//2]
        bb += pieces[pred]
        if (i+1) % 8 ==0:
            bb+="/"
    return boardtofen(bb[:-1])